In [46]:
import requests
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import csv
import os
from dotenv import load_dotenv
import time

# FMP News API Code

In [47]:
# loading in api file
load_dotenv(r"api.env")

# Obtaining key for API file
fmp_api_key = os.getenv("fmp_api_key")
if not fmp_api_key:
    raise ValueError("No API key set for fmp_api_key in .env file")

print("API key loaded successfully from .env file.")

API key loaded successfully from .env file.


In [48]:
# Setting up dictionaries in lists to facilitate looping through
stock_news = [
    {"symbol": "AMZN", "ipo_date": "1997-05-15"},
    {"symbol": "AAPL", "ipo_date": "1980-12-12"},
    {"symbol": "GOOG", "ipo_date": "2004-08-19"},
    {"symbol": "MSFT", "ipo_date": "1986-03-13"},
    {"symbol": "META", "ipo_date": "2012-05-18"},
    {"symbol": "NVDA", "ipo_date": "1999-01-22"}
]

press_releases = [
    {"company": "AMZN", "ipo_date": "1997-05-15"},
    {"company": "AAPL", "ipo_date": "1980-12-12"},
    {"company": "GOOG", "ipo_date": "2004-08-19"},
    {"company": "MSFT", "ipo_date": "1986-03-13"},
    {"company": "META", "ipo_date": "2012-05-18"},
    {"company": "NVDA", "ipo_date": "1999-01-22"}
]

historical_social_sentiment = [
    {"symbol": "AMZN", "ipo_date": "1997-05-15"},
    {"symbol": "AAPL", "ipo_date": "1980-12-12"},
    {"symbol": "GOOG", "ipo_date": "2004-08-19"},
    {"symbol": "MSFT", "ipo_date": "1986-03-13"},
    {"symbol": "META", "ipo_date": "2012-05-18"},
    {"symbol": "NVDA", "ipo_date": "1999-01-22"}
]

# Stock News Data Pull 

In [ ]:
# Obtaining today's date
current_date =date.today()

# Obtaining stock news data for each tech company
for news in stock_news:

    # setting up url path & parameter values
    url = f"https://financialmodelingprep.com/api/v3/stock_news?"
    page = 0
    limit = 2000

    # creating empty list to capture news data
    all_news = []

    # looping through pages of data to obtain stock news data
    while True:
        
        # applying parameter values to required params
        params = {
        "apikey" : fmp_api_key,
        "tickers" : news['symbol'],
        "page": page,
        'from': news['ipo_date'],
        'to': current_date,
        'limit': limit
        }

        # requesting data
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {news['symbol']} on page {page}: {e}")
            break

        # setting successfully requested data to variable data
        data = response.json()

        if not data:
            break

        # appending additional data to list
        all_news.extend(data)

        # moving to next page
        page += 1

    csv_file = f"{news['symbol']}_news_data.csv"
    headers = ["published_date", "Headline", "Brief", "URL"]
    formatted_data = []

    # writing all pulled news data to their respective companies in csv files
    if all_news:
        for record in all_news:

            formats = ['%Y-%m-%d %I:%M:%S %p', '%Y-%m-%d %H:%M:%S']

            if 'publishedDate' in record:
                for fmt in formats:
                    try:
                        record['publishedDate'] = datetime.strptime(record['publishedDate'], fmt).strftime('%d-%m-%Y')
                        break
                    except ValueError as e:
                        continue

            formatted_data.append({
            "published_date": record.get("publishedDate", ""),
            "Headline": record.get("title", ""),
            "Brief": record.get("text", ""),
            "URL": record.get("url", "")
            })

        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            writer.writeheader()
            writer.writerows(formatted_data)
        
        print(f"Data for {news['symbol']} written to {csv_file} with {len(formatted_data)} articles!")
    else:
        print(f"No data found for {news['symbol']}.")

print("All stock data successfully written to their respective CSV files.")
                
    

# Stock Press Release Data Pull

In [ ]:
# Obtaining stock press release data for each tech company
for press_release in press_releases:

    # setting up url path & parameter values
    url = f"https://financialmodelingprep.com/api/v3/press-releases/{press_release['company']}?"

    # setting page to zero
    page = 0

    # creating empty list to capture all press release data
    all_press_releases = []

    max_pages = 300

    # looping through pages of data to obtain stock press release data
    while page < max_pages:

        # applying parameter values to required params
        params = {
        "apikey" : fmp_api_key,
        "page": page
        }

        # requesting data
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {press_release['company']} on page {page}: {e}")
            break

        time.sleep(1)

        # setting successfully requested data to variable data
        data = response.json()
        
        print(f"Fetching data for {press_release['company']} - Page {page}...")

        if not data:
            break


        # appending additional data to list
        all_press_releases.extend(data)

        # moving to next page
        page += 1

    csv_file = f"{press_release['company']}_press_release_data.csv"
    headers = ["release_date", "Headline", "Brief"]
    formatted_data = []

    # writing all pulled press release data to their respective companies in csv files
    if all_press_releases:
        for record in all_press_releases:

            formats = ['%Y-%m-%d %I:%M:%S %p', '%Y-%m-%d %H:%M:%S']

            if 'date' in record:
                for fmt in formats:
                    try:
                        record['date'] = datetime.strptime(record['date'], fmt).strftime('%d-%m-%Y')
                        break
                    except ValueError as e:
                        continue

            formatted_data.append({
            "release_date": record.get("date", ""),
            "Headline": record.get("title", ""),
            "Brief": record.get("text", "")
            })

        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            writer.writeheader()
            writer.writerows(formatted_data)
        
        print(f"Data for {press_release['company']} written to {csv_file} with {len(formatted_data)} press releases!")
    else:
        print(f"No data found for {press_release['company']}.")

print("All stock data successfully written to their respective CSV files.")
                
    

# Text PreProcessing

In [51]:

# creating a function to clean text for VADER sentiment analysis
def text_cleaning(text, dataset):
    text = text.str.lower()
    text = text.str.replace(r"http\S+|www\S+", "", regex=True) # removing URLS
    text = text.str.replace(r"[^a-zA-Z0-9$%.,!?'\s-]", "", regex=True) # keeping relevant characters
    text = text.str.replace(r"\s+", " ", regex=True).str.strip() # removing extra spaces
    text = text.str.replace(r"\b(\d+)%", r"\1 percent", regex=True) # converting percentages
    text = text.str.replace(r"\b(\d+)M\b", r"\1 million", regex=True)  # Convert M to million
    text = text.str.replace(r"\b(\d+)B\b", r"\1 billion", regex=True)  # Convert B to billion

    # splitting texts manually based on ".!?"
    sentences = text.str.split(r'[.!?]\s+', regex=True)
    
    # creating column for the cleaned text and naming it 'cleaned'
    dataset['cleaned'] = sentences

    return dataset

In [52]:
# setting up lists to loop through for sentiment score and weighted scores analysis
stock_news_datasets = [
    "AMZN_news_data.csv",
    "AAPL_news_data.csv",
    "GOOG_news_data.csv",
    "MSFT_news_data.csv",
    "META_news_data.csv",
    "NVDA_news_data.csv"
]

news_press_releases_datasets = [
    "AMZN_press_release_data.csv",
    "AAPL_press_release_data.csv",
    "GOOG_press_release_data.csv",
    "MSFT_press_release_data.csv",
    "META_press_release_data.csv",
    "NVDA_press_release_data.csv"
]

In [ ]:
# creating a function to obtain sentiment scores
def sentiment_score_calculator(datasets_lists):

    # Initializing VADER sentiment analyzer
    analyzer = SentimentIntensityAnalyzer()


    # looping through datasets 
    for file in datasets_lists:
        
        # reading in file 
        news_press_release = pd.read_csv(file)

        # combining two text columns into one column to clean text in one go
        news_press_release['raw'] = news_press_release.apply(lambda row: row['Headline'] if pd.isna(row['Brief']) 
                                                        else (row['Brief'] if pd.isna(row['Headline']) 
                                                            else row['Headline'] + ". " + row['Brief']), axis=1)

        # cleaning text cleaning function with VADER appliation
        text_cleaning(news_press_release['raw'], news_press_release)


        # creating empty lists
        score_rating = []
        score = []

        # looping through each row in the column 'cleaned'
        for row in news_press_release['cleaned']:
            
            row_score = 0

            # looping through each sentence in current row and using vader to score each sentence
            for sentence in row:
                try:
                    print(sentence)
                    sentence_score = analyzer.polarity_scores(sentence)["compound"] # vader polarity score analyzer
                    print(sentence_score)
                    print("Done with sentence_score in row, onto next sentence.")

                    # accumulating each sentence's score for current row
                    row_score += sentence_score
                except RuntimeError as e:
                    print(f"Error processing sentence: {sentence}\n{e}")
                    continue

            # averaging row scores and appending to score
            avg_row_score = row_score / len(row)
            score.append(avg_row_score)

            print(f"row_score: {avg_row_score}")
            print(len(row))
            print("end of row, onto next row.")

            # classifying averaged row scores into various score ratings
            if avg_row_score > 0.05 and avg_row_score < 0.5:
                score_rating.append("weakly_positive")
                print("Weakly Positive")
            elif avg_row_score > 0.5:
                score_rating.append("strongly_positive")
                print("Strongly Positive")
            elif avg_row_score > -0.5 and avg_row_score < -0.05:
                score_rating.append("weakly_negative")
                print("Weakly Negative")
            elif avg_row_score < -0.5:
                score_rating.append("strongly_negative")
                print("Strongly Negative")
            else:
                score_rating.append("neutral")
                print("Neutral")

        # creating columns for sentiment and score
        news_press_release['sentiment'] = score_rating
        news_press_release['score'] = score

        # writing the updated vader sentiment scores to each file 
        news_press_release.to_csv(file, index=False)
        print(f"Finished updating {file} with sentiment and score.")

In [54]:
def aggregated_sentiments(datasets_lists):

    # looping through each dataset and applying aggregated sentiments
    for file in datasets_lists:

        # reading in dataset
        news_press_release = pd.read_csv(file)

        # Checking if either 'published_date' or 'release_date' exists and group by the first one found
        date_column = 'published_date' if 'published_date' in news_press_release.columns else 'release_date'

        # computing average score per unique publilshed_date
        avg_scores = news_press_release.groupby(date_column)['score'].mean().reset_index()

        # classifying weighted daily scores into classes

        # Defining conditions for classification
        conditions = [
            avg_scores['score'] > 0.5,
            (avg_scores['score'] > 0.05) & (avg_scores['score'] <= 0.5),
            (avg_scores['score'] >= -0.05) & (avg_scores['score'] <= 0.05),
            (avg_scores['score'] < -0.05) & (avg_scores['score'] >= -0.5),
            avg_scores['score'] < -0.5
        ]

        # Defining corresponding classifications
        classifications = [
            "strongly_positive",
            "weakly_positive",
            "neutral",
            "weakly_negative",
            "strongly_negative"
        ]

        # Assigning classifications based on conditions
        avg_scores['weighted_daily_sentiment'] = np.select(conditions, classifications, default="neutral")
        
        # merging to assign classifications to each row
        news_press_release = news_press_release.merge(avg_scores[[date_column, 'weighted_daily_sentiment']], on=date_column, how='left')

        # writing to csv files
        news_press_release.to_csv(file, index=False)
        print(f"Finished updating {file} with weighted daily sentiment.")


In [ ]:
# Calculating sentiments score for news and press release datasets for each company
sentiment_score_calculator(stock_news_datasets)
sentiment_score_calculator(news_press_releases_datasets)

# Calculating aggregated sentiments(daily) for news and press release datasets for each company
aggregated_sentiments(stock_news_datasets)
aggregated_sentiments(news_press_releases_datasets)